In [16]:
import re 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, WebDriverException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 구글 맵 페이지 열기 및 '뱅뱅막국수' 검색
driver.get("https://www.google.com/maps")
try:
    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

In [17]:
# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()

In [9]:
# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    scrollable_div = None
    for _ in range(5):  # 시도 횟수 조정 가능
        try:
            scrollable_div = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
            )
            break
        except:
            time.sleep(2)

    if scrollable_div is None:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        return

    # 스크롤을 끝까지 반복 수행
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for _ in range(30):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)
        
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:
            break
        last_height = new_height

scroll_down()

스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.


In [4]:
# # 리뷰 스크롤 다운 코드
# scroll_pause_time = 2  # 스크롤 후 대기 시간
# scroll_count = 10  # 스크롤 횟수 설정

# try:
#     # 리뷰를 담고 있는 스크롤 가능한 요소 선택
#     review_container = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.Ka9kIf.dS8AEf'))
#     )
    
#     for _ in range(scroll_count):
#         # 해당 요소 내에서 스크롤을 아래로 내림
#         driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_container)
#         time.sleep(scroll_pause_time)  # 스크롤 후 대기 시간
# except Exception as e:
#     print(f"스크롤 중 오류 발생: {e}")
#     driver.quit()


스크롤 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x00563913+24035]
	(No symbol) [0x004EBCA4]
	(No symbol) [0x003CC2D3]
	(No symbol) [0x0040DC86]
	(No symbol) [0x0040DECB]
	(No symbol) [0x0044B9D2]
	(No symbol) [0x0042FED4]
	(No symbol) [0x0044953F]
	(No symbol) [0x0042FC26]
	(No symbol) [0x0040218C]
	(No symbol) [0x0040310D]
	GetHandleVerifier [0x00809743+2800659]
	GetHandleVerifier [0x008642FE+3172302]
	GetHandleVerifier [0x0085CF12+3142626]
	GetHandleVerifier [0x00606CC0+692624]
	(No symbol) [0x004F4CBD]
	(No symbol) [0x004F19C8]
	(No symbol) [0x004F1B60]
	(No symbol) [0x004E4010]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DD809E+286]
	RtlGetAppContainerNamedObjectPath [0x77DD806E+238]



In [4]:
# 리뷰 데이터 수집
reviews_list = []

# 스크롤하면서 모든 리뷰를 로드
while True:
    # 리뷰 요소 선택
    reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
    
    # 각 리뷰 요소에서 데이터 수집
    for review in reviews:
        try:
            # '자세히' 버튼 클릭
            try:
                more_button = review.find_element(By.XPATH, './/span[2]/button')
                more_button.click()
                time.sleep(1)  # '자세히' 클릭 후 리뷰 로드 대기
            except NoSuchElementException:
                pass  # '자세히' 버튼이 없으면 무시하고 진행
            
            # 리뷰 작성자 ID
            reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
            # 리뷰 작성 날짜
            date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
            # 리뷰 내용
            review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
            reviews_list.append({
                "ID": reviewer_id,
                "Date": date,
                "Review": review_text
            })
        except Exception as e:
            print(f"리뷰 수집 중 오류: {e}")
            continue
    
    # 스크롤 다운
    driver.execute_script("arguments[0].scrollIntoView(true);", reviews[-1])
    time.sleep(2)  # 스크롤 후 로딩 대기

    # 새로운 리뷰가 로드되지 않으면 종료
    new_reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
    if len(new_reviews) == len(reviews):
        print("더 이상 로드할 리뷰가 없습니다.")
        break

# 데이터프레임으로 변환 및 출력
reviews_df = pd.DataFrame(reviews_list)
print(reviews_df)


더 이상 로드할 리뷰가 없습니다.
          ID  Date                                             Review
0      Amy J  2주 전  막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도....
1  Sujin Kim  3달 전  들기름막국수, 참기름막국수, 코다리막국수를 먹었는데 3개 메뉴 모두 맛있었어요. 국...
2        정의진  4달 전  유명한 곳이라 와봤는데 맛있고 친절하고 유명한 이유를 알겠네요^-^\n코다리랑 참기...
3  Sunny Lee  4달 전  강남 뱅뱅사거리에서 막국수로 유명한 맛집입니다. 들기름막국수 강추해요. 물막국수, ...
4     schnee  4달 전  여긴 진짜 참기름막국수!!!!!!!! 정말 맛있습니다. 막국수 별로 안좋아해서,, ...
5        박경호  4달 전  아주 맛있는 국수명인이 운영하는 가게\n막국수 맛있고 만두도 맛있어요.\n기본 반찬...
6   MIN JUNG  6달 전  김치만두가 너무너무 먹고싶어서, 집근처 맛집을 알아보던중 뱅뱅막국수에 방문했습니다....
7   Jisu Yoo  8달 전  사장님과 직원분들이 너무 친절하시고, 참기름막국수, 샤브샤브, 메밀전, 육전 주문했...
8        김우현  5달 전  음식 명장 & 명인 만든 음식이라고 기대를 많이 해서 그런지 물국수는 평범했음\n참...
9        박상욱  7달 전  봉평면메밀촌 시절부터 비빔막국수에 대한 극찬을 한 적이 있는데 이 집이 이렇게나 유...


In [15]:







# 리뷰 데이터 수집
reviews_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
for review in reviews:
    try:
        # '자세히' 버튼 클릭
        try:
            more_button = review.find_element(By.XPATH, './/span[2]/button')
            more_button.click()
            time.sleep(1)  # '자세히' 클릭 후 리뷰 로드 대기
        except NoSuchElementException:
            pass  # '자세히' 버튼이 없으면 무시하고 진행
        
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
        reviews_list.append({
            "ID": reviewer_id,
            "Date": date,
            "Review": review_text
        })
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

# 데이터프레임으로 변환 및 출력
reviews_df = pd.DataFrame(reviews_list)
print(reviews_df)

# CSV 파일로 저장
#reviews_df.to_csv('bangbang_makguksu_reviews.csv', index=False, encoding='utf-8-sig')

# 드라이버 종료
#driver.quit()

리뷰 수집 중 오류: Message: element click intercepted: Element <button class="w8nwRe kyuRq" aria-expanded="false" aria-controls="ChdDSUhNMG9nS0VJQ0FnSUNlMmRDZmdBRRAB" data-review-id="ChdDSUhNMG9nS0VJQ0FnSUNlMmRDZmdBRRAB" jslog="63707; track:click;metadata:WyIwYWhVS0V3aXJvdVczcE1lSkF4VlV5eFlGSFpPNUI4c1EwcE1GQ0FJb0FBIl0=" aria-label="더보기" jsaction="pane.wfvdle212.review.expandReview">...</button> is not clickable at point (198, 960). Other element would receive the click: <button class="Tya61d" data-photo-index="0" data-review-id="ChZDSUhNMG9nS0VJQ0FnSUNKck1TaU93EAE" data-tooltip="" jsaction="pane.wfvdle209.review.openPhoto;keydown:pane.wfvdle209.review.openPhoto;focus:pane.focusTooltip;blur:pane.blurTooltip" aria-label="im's k님의 리뷰에 포함된 1번째 사진" style="background-image: url(&quot;https://lh5.googleusercontent.com/p/AF1QipNQSkV-oItOefg9Oku_xgo7ysYJe_uFv3KGbs55=w300-h450-p-k-no&quot;); width: calc(50% - 1px); padding-top: 75%; margin-right: 2px;"></button>
  (Session info: chrome=130.0.6723.92)
S

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00563913+24035]
	(No symbol) [0x004EBCA4]
	(No symbol) [0x003CC2D3]
	(No symbol) [0x0040DC86]
	(No symbol) [0x0040DECB]
	(No symbol) [0x00403CE1]
	(No symbol) [0x0042FED4]
	(No symbol) [0x00403C04]
	(No symbol) [0x00430124]
	(No symbol) [0x0044953F]
	(No symbol) [0x0042FC26]
	(No symbol) [0x0040218C]
	(No symbol) [0x0040310D]
	GetHandleVerifier [0x00809743+2800659]
	GetHandleVerifier [0x008642FE+3172302]
	GetHandleVerifier [0x0085CF12+3142626]
	GetHandleVerifier [0x00606CC0+692624]
	(No symbol) [0x004F4CBD]
	(No symbol) [0x004F19C8]
	(No symbol) [0x004F1B60]
	(No symbol) [0x004E4010]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DD

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00563913+24035]
	(No symbol) [0x004EBCA4]
	(No symbol) [0x003CC2D3]
	(No symbol) [0x0040DC86]
	(No symbol) [0x0040DECB]
	(No symbol) [0x00403CE1]
	(No symbol) [0x0042FED4]
	(No symbol) [0x00403C04]
	(No symbol) [0x00430124]
	(No symbol) [0x0044953F]
	(No symbol) [0x0042FC26]
	(No symbol) [0x0040218C]
	(No symbol) [0x0040310D]
	GetHandleVerifier [0x00809743+2800659]
	GetHandleVerifier [0x008642FE+3172302]
	GetHandleVerifier [0x0085CF12+3142626]
	GetHandleVerifier [0x00606CC0+692624]
	(No symbol) [0x004F4CBD]
	(No symbol) [0x004F19C8]
	(No symbol) [0x004F1B60]
	(No symbol) [0x004E4010]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DD

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00563913+24035]
	(No symbol) [0x004EBCA4]
	(No symbol) [0x003CC2D3]
	(No symbol) [0x0040DC86]
	(No symbol) [0x0040DECB]
	(No symbol) [0x00403CE1]
	(No symbol) [0x0042FED4]
	(No symbol) [0x00403C04]
	(No symbol) [0x00430124]
	(No symbol) [0x0044953F]
	(No symbol) [0x0042FC26]
	(No symbol) [0x0040218C]
	(No symbol) [0x0040310D]
	GetHandleVerifier [0x00809743+2800659]
	GetHandleVerifier [0x008642FE+3172302]
	GetHandleVerifier [0x0085CF12+3142626]
	GetHandleVerifier [0x00606CC0+692624]
	(No symbol) [0x004F4CBD]
	(No symbol) [0x004F19C8]
	(No symbol) [0x004F1B60]
	(No symbol) [0x004E4010]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DD

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00563913+24035]
	(No symbol) [0x004EBCA4]
	(No symbol) [0x003CC2D3]
	(No symbol) [0x0040DC86]
	(No symbol) [0x0040DECB]
	(No symbol) [0x00403CE1]
	(No symbol) [0x0042FED4]
	(No symbol) [0x00403C04]
	(No symbol) [0x00430124]
	(No symbol) [0x0044953F]
	(No symbol) [0x0042FC26]
	(No symbol) [0x0040218C]
	(No symbol) [0x0040310D]
	GetHandleVerifier [0x00809743+2800659]
	GetHandleVerifier [0x008642FE+3172302]
	GetHandleVerifier [0x0085CF12+3142626]
	GetHandleVerifier [0x00606CC0+692624]
	(No symbol) [0x004F4CBD]
	(No symbol) [0x004F19C8]
	(No symbol) [0x004F1B60]
	(No symbol) [0x004E4010]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DD

...

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00563913+24035]
	(No symbol) [0x004EBCA4]
	(No symbol) [0x003CC2D3]
	(No symbol) [0x0040DC86]
	(No symbol) [0x0040DECB]
	(No symbol) [0x00403CE1]
	(No symbol) [0x0042FED4]
	(No symbol) [0x00403C04]
	(No symbol) [0x00430124]
	(No symbol) [0x0044953F]
	(No symbol) [0x0042FC26]
	(No symbol) [0x0040218C]
	(No symbol) [0x0040310D]
	GetHandleVerifier [0x00809743+2800659]
	GetHandleVerifier [0x008642FE+3172302]
	GetHandleVerifier [0x0085CF12+3142626]
	GetHandleVerifier [0x00606CC0+692624]
	(No symbol) [0x004F4CBD]
	(No symbol) [0x004F19C8]
	(No symbol) [0x004F1B60]
	(No symbol) [0x004E4010]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DD

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00563913+24035]
	(No symbol) [0x004EBCA4]
	(No symbol) [0x003CC2D3]
	(No symbol) [0x0040DC86]
	(No symbol) [0x0040DECB]
	(No symbol) [0x00403CE1]
	(No symbol) [0x0042FED4]
	(No symbol) [0x00403C04]
	(No symbol) [0x00430124]
	(No symbol) [0x0044953F]
	(No symbol) [0x0042FC26]
	(No symbol) [0x0040218C]
	(No symbol) [0x0040310D]
	GetHandleVerifier [0x00809743+2800659]
	GetHandleVerifier [0x008642FE+3172302]
	GetHandleVerifier [0x0085CF12+3142626]
	GetHandleVerifier [0x00606CC0+692624]
	(No symbol) [0x004F4CBD]
	(No symbol) [0x004F19C8]
	(No symbol) [0x004F1B60]
	(No symbol) [0x004E4010]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DD

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00563913+24035]
	(No symbol) [0x004EBCA4]
	(No symbol) [0x003CC2D3]
	(No symbol) [0x0040DC86]
	(No symbol) [0x0040DECB]
	(No symbol) [0x00403CE1]
	(No symbol) [0x0042FED4]
	(No symbol) [0x00403C04]
	(No symbol) [0x00430124]
	(No symbol) [0x0044953F]
	(No symbol) [0x0042FC26]
	(No symbol) [0x0040218C]
	(No symbol) [0x0040310D]
	GetHandleVerifier [0x00809743+2800659]
	GetHandleVerifier [0x008642FE+3172302]
	GetHandleVerifier [0x0085CF12+3142626]
	GetHandleVerifier [0x00606CC0+692624]
	(No symbol) [0x004F4CBD]
	(No symbol) [0x004F19C8]
	(No symbol) [0x004F1B60]
	(No symbol) [0x004E4010]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DD

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00563913+24035]
	(No symbol) [0x004EBCA4]
	(No symbol) [0x003CC2D3]
	(No symbol) [0x0040DC86]
	(No symbol) [0x0040DECB]
	(No symbol) [0x00403CE1]
	(No symbol) [0x0042FED4]
	(No symbol) [0x00403C04]
	(No symbol) [0x00430124]
	(No symbol) [0x0044953F]
	(No symbol) [0x0042FC26]
	(No symbol) [0x0040218C]
	(No symbol) [0x0040310D]
	GetHandleVerifier [0x00809743+2800659]
	GetHandleVerifier [0x008642FE+3172302]
	GetHandleVerifier [0x0085CF12+3142626]
	GetHandleVerifier [0x00606CC0+692624]
	(No symbol) [0x004F4CBD]
	(No symbol) [0x004F19C8]
	(No symbol) [0x004F1B60]
	(No symbol) [0x004E4010]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DD

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00563913+24035]
	(No symbol) [0x004EBCA4]
	(No symbol) [0x003CC2D3]
	(No symbol) [0x0040DC86]
	(No symbol) [0x0040DECB]
	(No symbol) [0x00403CE1]
	(No symbol) [0x0042FED4]
	(No symbol) [0x00403C04]
	(No symbol) [0x00430124]
	(No symbol) [0x0044953F]
	(No symbol) [0x0042FC26]
	(No symbol) [0x0040218C]
	(No symbol) [0x0040310D]
	GetHandleVerifier [0x00809743+2800659]
	GetHandleVerifier [0x008642FE+3172302]
	GetHandleVerifier [0x0085CF12+3142626]
	GetHandleVerifier [0x00606CC0+692624]
	(No symbol) [0x004F4CBD]
	(No symbol) [0x004F19C8]
	(No symbol) [0x004F1B60]
	(No symbol) [0x004E4010]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DD

In [ ]:
reviews_df

In [13]:
from selenium.webdriver.common.action_chains import ActionChains

# 리뷰 섹션의 컨테이너를 찾음
try:
    review_container = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde'))  # 리뷰 컨테이너의 CSS 선택자를 사용
    )
    
    # 스크롤을 반복적으로 내리기
    for _ in range(10):  # 원하는 만큼 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_container)
        time.sleep(2)  # 스크롤 후 로딩 대기

    print("스크롤 완료")
except Exception as e:
    print(f"스크롤 중 오류 발생: {e}")


스크롤 완료


In [7]:
# 리뷰 섹션의 컨테이너를 찾음
try:
    review_container = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde'))  # 리뷰 컨테이너의 CSS 선택자
    )
    
    # 스크롤을 반복적으로 내리기
    while True:
        # 현재 로드된 리뷰 수 확인
        reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        previous_count = len(reviews)
        
        # 스크롤을 아래로 내림
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_container)
        time.sleep(5)  # 스크롤 후 로딩 대기
        
        # 스크롤 후 로드된 리뷰 수 다시 확인
        reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        current_count = len(reviews)
        
        # 새로운 리뷰가 로드되지 않았으면 종료
        if current_count == previous_count:
            print("더 이상 로드할 리뷰가 없습니다.")
            break

    print("모든 리뷰를 끝까지 로드했습니다.")
except Exception as e:
    print(f"스크롤 중 오류 발생: {e}")


더 이상 로드할 리뷰가 없습니다.
모든 리뷰를 끝까지 로드했습니다.


In [18]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# 리뷰 섹션의 컨테이너를 찾음
try:
    review_container = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde'))
    )
    
    # 스크롤을 반복적으로 내리기
    max_wait_time = 5  # 최대 대기 시간 시작 (초)
    while True:
        # 현재 로드된 리뷰 수 확인
        reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        previous_count = len(reviews)
        
        # 스크롤을 아래로 내림
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_container)
        
        # 새로운 리뷰가 로드될 때까지 대기
        try:
            WebDriverWait(driver, max_wait_time).until(
                lambda driver: len(driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')) > previous_count
            )
            # 리뷰가 로드되면 대기 시간을 초기화
            max_wait_time = 5
        except TimeoutException:
            # 새로운 리뷰가 로드되지 않으면 더 이상 로드할 리뷰가 없는 것으로 간주하고 종료
            print("더 이상 로드할 리뷰가 없습니다.")
            break

        # 스크롤이 아래로 내려갈수록 로딩 시간이 길어질 수 있으므로, 실패 시 대기 시간 증가
        max_wait_time += 1  # 한 번 실패할 때마다 대기 시간을 1초씩 늘림 (필요 시 조정 가능)

    print("모든 리뷰를 끝까지 로드했습니다.")
except Exception as e:
    print(f"스크롤 중 오류 발생: {e}")


더 이상 로드할 리뷰가 없습니다.
모든 리뷰를 끝까지 로드했습니다.
